In [ ]:
# imports
import re
import time
import pickle
import logging
import gc
import os

import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt

from scipy import stats


log = logging.getLogger('log')
log.setLevel(logging.DEBUG)

lhnd = logging.StreamHandler()
lhnd.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
lhnd.setFormatter(formatter)

log.addHandler(lhnd)

In [ ]:
def lmap(f, arr):
    return list(map(f, arr))

def lfilter(f, arr):
    return list(filter(f, arr))

def foreach(it, f):
    for e in it:
        f(e)

In [ ]:
class Event:
    def __init__(self, ts, id):
        self.ts = int(ts)
        self.id = id
    
    def abs_id(self):
        return abs(self.id)
    
    def is_sub(self):
        return self.id > 0
    
    def __eq__(self, other):
        return self.raw_id() == other.raw_id() and self.ts == other.ts
    
    def __hash__(self):
        return self.raw_id() + ts * Math.pow()
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return '({}; {})'.format(self.ts, self.raw_id())

In [ ]:
total=947528

def raw_data_filter(file):
    # Mapping to events
    res = list()

    i = 0
    
    for line in file:
        cur = line.rstrip().split(',')
        cur = lmap(lambda p: (re.sub(';.*', '', p), re.sub('.*;', '', p)), cur)

        session = list()
        
        for j in range(0, len(cur)):
            try:
                session.append((cur[j][0], int(cur[j][1])))
            except ValueError:
                None
                
        res.append(session)

        i = i + 1
                
        if (i % 10000 == 0):
            gc.collect()

            log.debug("%d %% of mapping is done.", i / total * 100)

    
    return res

In [ ]:
chunks_count = 10

def split_mapping(raw, start, finish = chunks_count): 
    dexes = []
    
    for i in range(0, chunks_count):
        dexes.append(len(raw) // chunks_count * i)

    dexes.append(len(raw))
    
    log.debug(dexes)
        
    for i in range(start, finish):
        log.info('Processing range %d:%d', dexes[i], dexes[i + 1])
        
        filtered = raw_data_filter(raw[dexes[i]:dexes[i + 1]])
        
        fname = 'data/raw-chunk-' + str(i)
        
        gc.collect()
        
        with open(fname, 'wb') as f:
            pickle.dump(filtered, f, pickle.HIGHEST_PROTOCOL)
            
        log.info('Dumped to the file %s', fname)
        
        gc.collect()

def load_chuncks(start, finish = chunks_count):
    res = []
    
    for i in range(start, finish):
        start = time.time()
        
        fname = 'data/raw-chunk-' + str(i)
        
        with open(fname, 'rb') as f:
            res.append(pickle.load(f))
            
        log.debug("Loading finished in %d sec.", time.time() - start)
            
    return(res)



In [ ]:
data = raw_data_filter(open("data/sessions_public.txt","r"))
    


In [ ]:
total = dict()

for i in data:
    for j in i:
        if (j[1] in total.keys()):
            total[j[1]] = total[j[1]] + 1
        else:
            total[j] = 0